In [3]:
from typing import Any, Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchResults
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.1)

class StockMarketSymbolSearchToolArgsSchema(BaseModel):
  query: str = Field(description="The query you will search for. Example query: Stock Market Symbol for Apple Company")

class StockMarketSymbolSearchTool(BaseTool):
  name = "StockMarketSymbolSearchTool"
  description= """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
  args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema
  
  def _run(self, query):
    ddg = DuckDuckGoSearchResults()
    return ddg.run(query)

    

agent = initialize_agent(
  llm=llm, 
  verbose=True,
  agent=AgentType.OPENAI_FUNCTIONS,
  handle_parsing_errors=True,
  tools=[StockMarketSymbolSearchTool()]
)

prompt = "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have."

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock market symbol'}`


[snippet: Find the latest Cloudflare, Inc. (NET) stock quote, history, news and other vital information to help you with your stock trading and investing. ... Market Cap: 32.753B: Beta (5Y Monthly) 1.13 ..., title: Cloudflare, Inc. (NET) Stock Price, News, Quote & History - Yahoo Finance, link: https://finance.yahoo.com/quote/NET/], [snippet: Get the latest Cloudflare Inc (NET) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions., title: Cloudflare Inc (NET) Stock Price & News - Google Finance, link: https://www.google.com/finance/quote/NET:NYSE], [snippet: Technology Expedia, Cloudflare, Affirm, PepsiCo, AMC Networks, Pinterest, Take-Two, and More of Today's Stock Market Movers Feb. 9, 2024 at 4:49 p.m. ET by Barron's, title: NET Stock Price | Cloudflare Inc. Cl

{'input': "Give me information on Cloudflare's stock and help me analyze if it's a potential good investment. Also tell me what symbol does the stock have.",
 'output': "Here are some resources where you can find information on Cloudflare's stock price, history, news, and other vital information:\n\n1. [Cloudflare, Inc. (NET) Stock Price, News, Quote & History - Yahoo Finance](https://finance.yahoo.com/quote/NET/)\n2. [Cloudflare Inc (NET) Stock Price & News - Google Finance](https://www.google.com/finance/quote/NET:NYSE)\n3. [NET Stock Price | Cloudflare Inc. Cl A Stock Quote (U.S.: NYSE) - MarketWatch](https://www.marketwatch.com/investing/stock/net)\n4. [Cloudflare (NET) Stock Price, News & Info | The Motley Fool](https://www.fool.com/quote/nyse/net)\n\nYou can review these sources to gather more information on Cloudflare's stock performance and financial data. This information can help you analyze if Cloudflare's stock is a potential good investment."}